# Regionalization: Hydrological modeling using LSTMs

In [ ]:
# general
from tqdm import tqdm
from pathlib import Path
from glob import glob
import pandas as pd
import shutil
import yaml 
import pickle
import os

# geospatial
import xarray as xr
import geopandas as gpd

# hydrology
from neuralhydrology.nh_run import start_run
from neuralhydrology.nh_run import eval_run
from neuralhydrology.evaluation import metrics
from neuralhydrology.evaluation import get_tester
from neuralhydrology.utils.config import Config

import torch
torch.cuda.is_available()

os.chdir('/home/rooda/OneDrive/Projects/DeepHydro/')
path_disk = "/home/rooda/Pipeline/DeepHydro/NEURAL"

In [ ]:
encode_Q = {'Q': {'dtype': 'int32', 'scale_factor': 0.0001, '_FillValue': -9999}}

def pickle_to_nc(pickle_path):
    
    with open(pickle_path, 'rb') as f:
        x = pickle.load(f);

        df = []
        for basin in tqdm(x.keys(), leave = False): 
        
            dataset_xr = x[basin]['1D']['xr']
            dataset_xr = dataset_xr.isel(time_step=0).drop_vars('time_step')
            dataset_xr = dataset_xr.PMET_q_mm_sim.clip(0)
            df.append(dataset_xr)

    df = xr.concat(df, dim = "basin_id")
    df = df.assign_coords({"basin_id": list(x.keys())})
    df = df.rename("Q")
    return df

In [ ]:
q_metadata_pmet  = pd.read_csv("data/Attributes_all_basins_pmet.csv", index_col = 0)
q_metadata_pmet  = q_metadata_pmet.loc[gpd.read_file("data/GIS/Basins_PMETobs_points_subset.gpkg").gauge_id]
q_metadata = pd.read_csv("data/Attributes_all_basins.csv", index_col = 0)

In [ ]:
# pmet basins
with open("modelling/basins_id_train.txt", "w") as file: # train
    for x in q_metadata_pmet.index:
        file.write(x + "\n")

shutil.copyfile("modelling/basins_id_train.txt",  # validation = train
                "modelling/basins_id_validation.txt")

# all basins
with open("modelling/basins_id_test.txt", "w") as file: # test
    for x in q_metadata.index:
        file.write(x + "\n")

## Historical run

### OGGM on

In [ ]:
# remove previous runs
[shutil.rmtree(d) for d in glob(path_disk + '/runs/historical_ALL_OGGM_on*')];

# Set Hydro_NH_setup 
with open('modelling/Hydro_NH_setup.yml') as stream:
    data = yaml.safe_load(stream)

    # experiment name + data
    data['experiment_name'] = "historical_ALL_OGGM_on"
    data['data_dir']        = path_disk + "/data/historical_ALL"
    data['dynamic_inputs']  = ["PMET_precip_full_mm", "PMET_tmax_full_degC", "PMET_tmin_full_degC", "PMET_pet_full_mm", "PMET_glacier_melt_mm"]

    # dates
    data['train_start_date']      = "01/01/2000"
    data['train_end_date']        = "31/12/2019"
    data['validation_start_date'] = "01/01/2000"
    data['validation_end_date']   = "31/12/2019"
    data['test_start_date']       = "01/01/2000"
    data['test_end_date']         = "31/12/2019"

    # hyperparameters 
    data['epochs']          = 5
    data['batch_size']      = 256
    data['hidden_size']     = 128
    data['learning_rate']   = 0.005
    data['output_dropout']  = 0.4
    data['seq_length']      = 365

with open('modelling/Hydro_NH_setup.yml', 'w') as stream:
    yaml.dump(data, stream, default_flow_style=False)

torch.cuda.empty_cache()
start_run(config_file=Path('modelling/Hydro_NH_setup.yml'))

# generature test_results.p
path_base = glob(path_disk + "/runs/historical_ALL_OGGM_on*/", recursive = True)[0]
tester = get_tester(cfg=Config(Path(path_base + "config.yml")), run_dir=Path(path_base), period="test", init_model=True)
tester = tester.evaluate(epoch=data['epochs'] , save_results=True, save_all_output=False, metrics=False)

df = pickle_to_nc(glob(path_base + "test/*/test_results.p")[0])
df.to_netcdf("results/runoff/total_runoff_historical_LSTM_OGGM_on_all.nc", encoding = encode_Q)
(df.resample(date="YS").sum().mean("date") * q_metadata.total_area * 1e6 / (365 * 1e3 * 86400)).sum()

### OGGM off

In [ ]:
# remove previous runs
[shutil.rmtree(d) for d in glob(path_disk + '/runs/historical_ALL_OGGM_off*')];

# Set Hydro_NH_setup 
with open('modelling/Hydro_NH_setup.yml') as stream:
    data = yaml.safe_load(stream)

    # experiment name + data
    data['experiment_name'] = "historical_ALL_OGGM_off"
    data['data_dir']        = path_disk + "/data/historical_ALL"
    data['dynamic_inputs']  = ["PMET_precip_full_mm", "PMET_tmax_full_degC", "PMET_tmin_full_degC", "PMET_pet_full_mm"]

    # dates
    data['train_start_date']      = "01/01/2000"
    data['train_end_date']        = "31/12/2019"
    data['validation_start_date'] = "01/01/2000"
    data['validation_end_date']   = "31/12/2019"
    data['test_start_date']       = "01/01/2000"
    data['test_end_date']         = "31/12/2019"

    # hyperparameters 
    data['epochs']          = 5
    data['batch_size']      = 256
    data['hidden_size']     = 128
    data['learning_rate']   = 0.005
    data['output_dropout']  = 0.4
    data['seq_length']      = 365
    
with open('modelling/Hydro_NH_setup.yml', 'w') as stream:
    yaml.dump(data, stream, default_flow_style=False)

torch.cuda.empty_cache()
start_run(config_file=Path('modelling/Hydro_NH_setup.yml'))

# generature test_results.p
path_base = glob(path_disk + "/runs/historical_ALL_OGGM_off*/", recursive = True)[0]
tester = get_tester(cfg=Config(Path(path_base + "config.yml")), run_dir=Path(path_base), period="test", init_model=True)
tester = tester.evaluate(epoch=data['epochs'], save_results=True, save_all_output=False, metrics=False)

df = pickle_to_nc(glob(path_base + "test/*/test_results.p")[0])
df.to_netcdf("results/runoff/total_runoff_historical_LSTM_OGGM_off_all.nc", encoding = encode_Q)
(df.resample(date="YS").sum().mean("date") * q_metadata.total_area * 1e6 / (365 * 1e3 * 86400)).sum()

## Future run

In [ ]:
start_date = "01/01/2022" # data starts from 2021, but initialisation requires one year of data
end_date   = "31/12/2098"
gcm_list   = ["GFDL-ESM4", "IPSL-CM6A-LR", "MIROC6", "MPI-ESM1-2-LR", "MRI-ESM2-0"]
ssp_list   = ["ssp126", "ssp585"]

### OGGM on

In [ ]:
path_base = glob(path_disk + "/runs/historical_ALL_OGGM_on*/", recursive = True)[0]

for gcm in gcm_list:
    for ssp in ssp_list: 

        # trick to avoid training again
        with open(path_base + "config.yml") as stream:
            data = yaml.safe_load(stream)

            data['data_dir'] = path_disk + "/data/future_ALL_" + gcm + "_" + ssp
            data['test_start_date'] = start_date
            data['test_end_date'] = end_date

        with open(path_base + "config.yml", 'w') as stream:
            yaml.dump(data, stream, default_flow_style=False)

        tester = get_tester(cfg=Config(Path(path_base + "config.yml")), run_dir=Path(path_base), period="test", init_model=True)
        tester = tester.evaluate(epoch=data['epochs'], save_results=True, save_all_output=False, metrics=False)

        df = pickle_to_nc(glob(path_base + "test/*/test_results.p")[0])
        df.to_netcdf("results/runoff/total_runoff_future_{}_{}_LSTM_OGGM_on_all.nc".format(gcm, ssp), encoding = encode_Q)

### OGGM off

In [ ]:
path_base = glob(path_disk + "/runs/historical_ALL_OGGM_off*/", recursive = True)[0]

for gcm in gcm_list:
    for ssp in ssp_list: 

        # trick to avoid training again
        with open(path_base + "config.yml") as stream:
            data = yaml.safe_load(stream)

            data['data_dir'] = path_disk + "/data/future_ALL_" + gcm + "_" + ssp
            data['test_start_date'] = start_date
            data['test_end_date'] = end_date

        with open(path_base + "config.yml", 'w') as stream:
            yaml.dump(data, stream, default_flow_style=False)

        tester = get_tester(cfg=Config(Path(path_base + "config.yml")), run_dir=Path(path_base), period="test", init_model=True)
        tester = tester.evaluate(epoch=data['epochs'], save_results=True, save_all_output=False, metrics=False)

        df = pickle_to_nc(glob(path_base + "test/*/test_results.p")[0])
        df.to_netcdf("results/runoff/total_runoff_future_{}_{}_LSTM_OGGM_off_all.nc".format(gcm, ssp), encoding = encode_Q)